## LLMをプログラム的に操作する

これまでに ChatGPT のような大規模言語モデル (LLM) とやり取りしたことがあるはずです。これは通常、UI またはアプリケーションを通じて行われます。

このノートブックでは、Python を使用して、API を介して直接 LLM に接続し、クエリを実行します。このラボでは、**OLLAMA Mistral-7B** モデルを選択しました。[(https://ollama.com/library/mistral](https://ollama.com/library/mistral)). これは完全にオープン ソース モデル (Apache 2.0 ライセンス) であり、他の商用モデルやオープン ソース モデルよりもはるかに軽量ですが、特に私たちが使用しようとしているタスクでは非常に優れています。

このモデルは既にラボ クラスターにデプロイされています。これは、モデルが小さい場合でも GPU ではなく CPU でホストしているため、LLM サービスの速度は低下しますが、デモ環境用の低コスト オプションが提供されるためです。理想的には、少なくとも 24 GB の RAM を搭載した GPU が必要になります。

In [1]:
!pip install -q langchain==0.1.14


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


### 要件とインポート

ラボの指示に従って起動する適切なワークベンチ イメージを選択した場合は、必要なライブラリがすべてすでに用意されているはずです。そうでない場合は、次のセルの最初の行のコメントを解除して、適切なパッケージをすべてインストールします。その後、必要なライブラリをインポートします。

In [2]:
# Imports
import json
import os
from os import listdir
from os.path import isfile, join
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain_community.llms import Ollama
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

### Langchain

Langchain (https://www.langchain.com/) は、言語モデルを利用したアプリケーションを開発するためのフレームワークです。LLM を適切にクエリするために手動で記述する必要のあるすべての定型コードを処理します。

まず、LLM API をクエリできる場所とモデルに適用されるいくつかのパラメータで定義される **llm** インスタンスを作成します。たとえば、`max_new_tokens` は、モデルに最大 96 個のトークン (単語または単語の一部) で応答するように指示します。ここで非常に低く設定されている `temperature` は、モデルに真実に基づいたままで、あまり "創造的" にならないように指示します。結局のところ、ここでは派手な詩を書こうとしているわけではありません。

In [3]:
inference_server_url = "http://llm.ic-shared-llm.svc.cluster.local:11434"

In [4]:
# LLM definition
llm = Ollama(
    base_url=inference_server_url,
    model="mistral",
    top_p=0.92,
    temperature=0.01,
    num_predict=512,
    repeat_penalty=1.03,
    callbacks=[StreamingStdOutCallbackHandler()]
)

print(llm)

Ollama
Params: {'model': 'mistral', 'format': None, 'options': {'mirostat': None, 'mirostat_eta': None, 'mirostat_tau': None, 'num_ctx': None, 'num_gpu': None, 'num_thread': None, 'num_predict': 512, 'repeat_last_n': None, 'repeat_penalty': 1.03, 'temperature': 0.01, 'stop': None, 'tfs_z': None, 'top_k': None, 'top_p': 0.92}, 'system': None, 'template': None, 'keep_alive': None}


また、モデルに送信するすべてのリクエスト ("Prompt") に適用する **テンプレート** も必要です。

モデルにクエリを実行する場合、ユーザーが入力した内容をそのまま送信することはほとんどないでしょう。この入力に加えて、モデルがそれをどのように処理するかを認識できるように、適切な指示を与える必要があります。何をどのように回答するか、何に回答してはいけないか、どのような口調で答えるかなどです。

In [5]:
template="""<s>[INST]<<SYS>>
You are a helpful, respectful and honest assistant. Always be as helpful as possible, while being safe.
You will be asked a question, to which you must give an answer.
Your answer should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.
Please ensure that your responses are socially unbiased and positive in nature.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.
If you don't know the answer to a question, answer "I don't know".
<</SYS>>

### QUESTION:
{input}

### ANSWER:
[/INST]
"""
PROMPT = PromptTemplate(input_variables=["input"], template=template)

Langchain を使用すると、これらの要素を簡単に "つなぎ合わせ"、モデルのクエリに使用する **会話** オブジェクトを作成できるようになりました。

In [6]:
conversation = LLMChain(llm=llm,
                        prompt=PROMPT,
                        verbose=False
                        )

これで、モデルをクエリする準備ができました。クエリからの出力を待ちます。LLM サービスでは GPU ではなく CPU を使用しているため、1 ～ 2 分かかります。

In [7]:
query = "What is Artificial Intelligence?"
conversation.predict(input=query); # ";" at the end of the line hides final output (repetion of the streamed answer)

 Artificial Intelligence (AI) refers to the simulation of human intelligence in machines that are programmed to think and learn like humans. Rather than being limited by their pre-programmed instructions, AI systems are designed to adapt and improve from experience. They can perform tasks that typically require human intelligence, such as visual perception, speech recognition, decision-making, and language translation. AI systems can be categorized as either narrow or general. Narrow AI is designed to perform a specific task, such as playing chess or recognizing speech, while general AI, also known as artificial general intelligence, has the ability to understand, learn, and apply knowledge across a wide range of tasks at a level equal to or beyond a human being.